# intermediate_XED_binary
This notebook takes our custom XED binary dataset and trains an intermediate model.

## Imports & Settings

First, update working directory to parent so that we may use our custom functions

In [1]:
import os
os.chdir('..')
# os.getcwd( )

In [2]:
import params
from utils import *
from trainer import *

import numpy as np
import pandas as pd
from datasets import load_from_disk

from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

# suppress model warning
from transformers import logging
logging.set_verbosity_error()

# set logging level
import logging
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [3]:
# set general seeds
set_seeds(1)

# set dataloader generator seed
g = torch.Generator()
g.manual_seed(1)

# set params for this model
params.num_labels = 2
params.output_dir = "model_saves/intermediate_XED_binary_01"
params.dataset_path = "data/inter_XED/itesd_xed_binary_balanced.hf"

# Ensure we're on an ARM environment if necessary.
platform_check()

We're Armed: macOS-13.1-arm64-i386-64bit


## Load Data

### Binary XED

In [4]:
datasets = load_from_disk(params.dataset_path)
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'num_word_text'],
        num_rows: 8096
    })
    validation: Dataset({
        features: ['text', 'label', 'num_word_text'],
        num_rows: 2400
    })
    test: Dataset({
        features: ['text', 'label', 'num_word_text'],
        num_rows: 1655
    })
})

In [5]:
# we will need to view and prep the datasets
# this is more easily done as dataframes
train_df = datasets['train'].to_pandas()
validate_df = datasets['validation'].to_pandas() 

In [6]:
# view training dataset
print("train_df Info:")
print(train_df.info())
print("\ntrain_df Value Counts")
print(train_df['label'].value_counts())

train_df Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8096 entries, 0 to 8095
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           8096 non-null   object
 1   label          8096 non-null   int64 
 2   num_word_text  8096 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 189.9+ KB
None

train_df Value Counts
0    4048
1    4048
Name: label, dtype: int64


In [7]:
# view validation dataset
print("validate_df Info:")
print(validate_df.info())
print("\n validate_df Value Counts")
print(validate_df['label'].value_counts())

validate_df Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           2400 non-null   object
 1   label          2400 non-null   int64 
 2   num_word_text  2400 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 56.4+ KB
None

 validate_df Value Counts
1    1229
0    1171
Name: label, dtype: int64


## Preprocess

In [8]:
train_token_ids, train_attention_masks = encode_text(train_df.text.values)
validate_token_ids, validate_attention_masks = encode_text(validate_df.text.values)

In [9]:
train_features = []
for i in range(len(train_token_ids)):
    train_features.append({'label': train_df.label.values[i], 
                           'input_ids': train_token_ids[i], 
                           'attention_mask':train_attention_masks[i]})

validate_features = []
for i in range(len(validate_token_ids)):
    validate_features.append({'label': validate_df.label.values[i], 
                              'input_ids': validate_token_ids[i],
                              'attention_mask':validate_attention_masks[i]})

## Data Split
We split the dataset into train (80%) and validation (20%) sets, and wrap them around a torch.utils.data.DataLoader object.

In [10]:
# Prepare DataLoader
train_dataloader = DataLoader(
            train_features,
            sampler = RandomSampler(train_features),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
            collate_fn=seq_class_collate
        )

validation_dataloader = DataLoader(
            validate_features,
            sampler = RandomSampler(validate_features),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
            collate_fn=seq_class_collate
        )

In [11]:
# view an example from the dataloader
next(iter(train_dataloader))

[tensor([[    0,   133, 11036,   313,  1516, 10789,     7,  1930,    39,    86,
             19, 38201,   479,     2,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1],
         [    0,   100,   437,  1227,   479,     2,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1],
         [    0,   970, 12960,     7,    28,   103,  1666,     2,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1],
         [    0,  1185,  2662,     8,  2268,   110,  1612,

## Train

Download transformers.RobertaForSequenceClassificatio, which is a RoBERTa model with a linear layer for sentence classification (or regression) on top of the pooled output:

In [12]:
# Load the RobertaForSequenceClassification model
model = RobertaForSequenceClassification.from_pretrained('roberta-base',
                                                         num_labels = params.num_labels,
                                                         output_attentions = False,
                                                         output_hidden_states = False,
                                                         )

from torchinfo import summary
summary(model, input_size=(1, 512), dtypes=['torch.IntTensor'])

Layer (type:depth-idx)                                       Output Shape              Param #
RobertaForSequenceClassification                             [1, 2]                    --
├─RobertaModel: 1-1                                          [1, 512, 768]             --
│    └─RobertaEmbeddings: 2-1                                [1, 512, 768]             --
│    │    └─Embedding: 3-1                                   [1, 512, 768]             38,603,520
│    │    └─Embedding: 3-2                                   [1, 512, 768]             768
│    │    └─Embedding: 3-3                                   [1, 512, 768]             394,752
│    │    └─LayerNorm: 3-4                                   [1, 512, 768]             1,536
│    │    └─Dropout: 3-5                                     [1, 512, 768]             --
│    └─RobertaEncoder: 2-2                                   [1, 512, 768]             --
│    │    └─ModuleList: 3-6                                  --               

Set model to device, initialize trainer

In [13]:
model.to(params.device)
print(f"Device: {params.device}")

optimizer = torch.optim.Adam(params=model.parameters(), 
                             lr=params.learning_rate,
                             weight_decay=params.weight_decay) #roberta

trainer = Trainer(model=model,
                  device=params.device,
                  tokenizer=params.tokenizer,
                  train_dataloader=train_dataloader,
                  validation_dataloader=validation_dataloader,
                  epochs=params.epochs,
                  optimizer=optimizer,
                  val_loss_fn=params.val_loss_fn,
                  num_labels=params.num_labels,
                  output_dir=params.output_dir,
                  save_freq=params.save_freq,
                  checkpoint_freq=params.checkpoint_freq)

output_parameters()

Device: mps

      Training Dataset: data/inter_XED/itesd_xed_binary_balanced.hf
      Number of Labels: 2
      Batch Size: 16
      Learning Rate: 1e-05
      Weight Decay: 0.01
      Epochs: 10
      Output Directory: model_saves/intermediate_XED_binary_01
      Save Frequency: 1
      Checkpoint Frequency: 1
      Max Length: 256
      


Fit the model to our training data.

In [14]:
trainer.fit()

	 Validation 149: 100%|██████████| 150/150 [00:09<00:00, 15.79batch/s]

 	 - Train loss: 0.537541
	 - Validation Loss: 0.458887
	 - Validation Accuracy: 0.802500
	 - Validation F1: 0.813144
	 - Validation Recall: 0.888311
	 - Validation Precision: 0.762801 

	 * Model @ epoch 1 saved to model_saves/intermediate_XED_binary_01/E01_A0.8_F0.81
	 * Model checkpoint saved to model_saves/intermediate_XED_binary_01/E01_A0.8_F0.81/checkpoint.pt
	 Validation 149: 100%|██████████| 150/150 [00:08<00:00, 16.95batch/s]

 	 - Train loss: 0.393076
	 - Validation Loss: 0.445748
	 - Validation Accuracy: 0.822083
	 - Validation F1: 0.818350
	 - Validation Recall: 0.825355
	 - Validation Precision: 0.830261 

	 * Model @ epoch 2 saved to model_saves/intermediate_XED_binary_01/E02_A0.82_F0.82
	 * Model checkpoint saved to model_saves/intermediate_XED_binary_01/E02_A0.82_F0.82/checkpoint.pt
	 Validation 149: 100%|██████████| 150/150 [00:08<00:00, 16.84batch/s]

 	 - Train loss: 0.322635
	 - Validation Loss: